In [28]:
import gensim.downloader as api
import os
import numpy as np

Ucitavanje dataseta

In [ ]:
dataset = api.load('text8')

[==================================================] 100.0% 31.6/31.6MB downloaded


Uzimanje privh 1000 reci kao sample, 
Pravljenje vokabulara i njegovog inverza.
{'anarchism': 0, 'originated': 1...}
{0: 'anarchism', 1: 'originated'...}

In [ ]:
import collections

sample = next((iter(dataset)))[:1000]
count = collections.Counter(sample)

vocab = {}
inverse_vocab = {}

for word in count:
    vocab[word] = len(vocab)
    inverse_vocab[len(inverse_vocab)] = word
 
print(vocab)
print(inverse_vocab)

{'anarchism': 0, 'originated': 1, 'as': 2, 'a': 3, 'term': 4, 'of': 5, 'abuse': 6, 'first': 7, 'used': 8, 'against': 9, 'early': 10, 'working': 11, 'class': 12, 'radicals': 13, 'including': 14, 'the': 15, 'diggers': 16, 'english': 17, 'revolution': 18, 'and': 19, 'sans': 20, 'culottes': 21, 'french': 22, 'whilst': 23, 'is': 24, 'still': 25, 'in': 26, 'pejorative': 27, 'way': 28, 'to': 29, 'describe': 30, 'any': 31, 'act': 32, 'that': 33, 'violent': 34, 'means': 35, 'destroy': 36, 'organization': 37, 'society': 38, 'it': 39, 'has': 40, 'also': 41, 'been': 42, 'taken': 43, 'up': 44, 'positive': 45, 'label': 46, 'by': 47, 'self': 48, 'defined': 49, 'anarchists': 50, 'word': 51, 'derived': 52, 'from': 53, 'greek': 54, 'without': 55, 'archons': 56, 'ruler': 57, 'chief': 58, 'king': 59, 'political': 60, 'philosophy': 61, 'belief': 62, 'rulers': 63, 'are': 64, 'unnecessary': 65, 'should': 66, 'be': 67, 'abolished': 68, 'although': 69, 'there': 70, 'differing': 71, 'interpretations': 72, 'what

Pravljenje windowa(indeksiranje, tj. pravljenje parova po ID)

In [ ]:
window_size = 5
training_pairs = []

for i in range(len(sample)):
    #uzimamo id reci
    center_word_id = vocab[sample[i]]

    start = max(0, i - window_size)
    end = min(i + window_size + 1, len(sample))

    for j in range(start, end):
        if i == j:
            continue

        pair_word_id = vocab[sample[j]]
        training_pairs.append((center_word_id, pair_word_id))

print(f"Ukupno parova: {len(training_pairs)}")
print(f"Prvih 20 parova (ID, ID): {training_pairs[:20]}")       

Ukupno parova: 9970
Prvih par parova (ID, ID): [(5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6)]


Pravljenje matrica w1 i w2, i postavljanje limita putem xavier inicijalizacije

In [30]:
vocab_size = len(vocab)
vector_size = 10

limit = np.sqrt(6 / (vocab_size + vector_size))

W1 = np.random.uniform(-limit, limit, (vocab_size, vector_size))
W2 = np.random.uniform(-limit, limit, (vector_size, vocab_size))


print(limit)
print(f"W1 (Vokabular x Vektor): {W1.shape}")
print(f"W2 (Vektor x Vokabular): {W2.shape}")


0.11521430589638529
W1 (Vokabular x Vektor): (442, 10)
W2 (Vektor x Vokabular): (10, 442)


FORWARD PASS    

In [ ]:
def softmax(x):
    return np.exp(x) / sum(np.exp(x))

def forward_pass(word_id, W1, W2):
    # h: embedding za word (hidden layer)
    h = W1[word_id]
    # u: dot product (output)
    u = np.dot(h,W2)

    y_pred = softmax(u)

    return y_pred, h

BACKPROPAGATION

In [ ]:
def backpropagation(error, h ,word_id, W1, W2, learning_rate):
    # gradijent za W2 (dimenzije cele matrice)
    dW2 = np.outer(h, error)
    # gradijent za W1 (dimenzije jednog vektora(reda) u matrici)
    dW1 = np.dot(error, W2.T)

    W2 -= learning_rate * dW2
    W1[word_id] -= learning_rate * dW1

    return W1, W2

Glavna petlja   

In [ ]:
epochs = 50
learning_rate = 0.01

for epoch in range(epochs):
    loss = 0

    for center_word_id, pair_word_id in training_pairs:
        y_pred, h = forward_pass(center_word_id, W1, W2)

        error = np.copy(y_pred)
        error[pair_word_id] -= 1

        W1, W2 = backpropagation(error, h, center_word_id, W1, W2, learning_rate)

        loss += -np.log(y_pred[pair_word_id])

    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs} | Loss: {loss:.4f}")

